In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os

In [33]:
com = open ('./Complejidades de Vy t')
rk = com.read()

In [34]:
rk = rk.replace("Archivo & Serie temporal Complejidad Media c y c/b & Serie temporal Complejidad Mediana c y c/b & Serie diferencia temporal Complejidad Media c y c/b & Serie diferencia temporal Complejidad Mediana c y c/b", "")

In [35]:
rk = rk.split('\n')

In [142]:
resultados_kolgomorov = {}
for fila in rk[1:]:
    nombre_del_archivo, com_M_V, com_m_V, com_M_T, com_m_T = fila.split("&")
    resultados_kolgomorov[nombre_del_archivo.strip()] = {'com_M_V':float(com_M_V.split(',')[1].strip()), 'com_m_V': float(com_m_V.split(',')[1].strip()), 'com_M_T':float(com_M_T.split(',')[1].strip()), 'com_m_T':float(com_m_T.split(',')[1].strip())}

In [143]:
resultados_pv_y_lyap = {}
for fe in os.scandir('falsos vecinos'):
    if 'png' not in fe.path and 'txt' not in fe.path:
        archivo = fe.name.split('.bin')[0]
        if not archivo in resultados_pv_y_lyap:
            resultados_pv_y_lyap[archivo] = {'V': {}, 't': {}}
        V_o_t = fe.name.split('maximos_')[1][0]
        fv_o_lyap = fe.name.split('maximos_')[1][1:]
        if fv_o_lyap == 'fv':
            test = np.loadtxt(fe.path, delimiter=',')
            resultados_pv_y_lyap[archivo][V_o_t]['fv'] = {'x': test[:,0], 'y': test[:,1]}
        elif fv_o_lyap == 'lyap':
            test = np.loadtxt(fe.path, delimiter=',')
            resultados_pv_y_lyap[archivo][V_o_t]['lyap'] = {'exp': test[1:-1], 'sum': sum(test[1:-1])}

In [161]:
for archivo in resultados_pv_y_lyap:
    print(archivo)
    for serie in resultados_pv_y_lyap[archivo]:
        x = resultados_pv_y_lyap[archivo][serie]['fv']['x']
        y = resultados_pv_y_lyap[archivo][serie]['fv']['y']
        plt.plot(x,y*100, '.-b')
        plt.grid(True)
        plt.xlabel(r'Dimension', fontsize=14)
        plt.ylabel(r'False nearest neighbours (%)', fontsize=14)
        i = check_dE(y)
        if i:
            plt.plot(x[i-1],y[i-1]*100, '*r')
            plt.text(8,80, f"La dimension de embedding es : {i}")
        #coef_lyap = "Coef Lyap: "+ str(resultados_pv_y_lyap[archivo][serie]['lyap']['exp'][:])
        #plt.text(8,80,coef_lyap)
        suma = np.format_float_scientific(resultados_pv_y_lyap[archivo][serie]['lyap']['sum'], unique=False, precision=4)
        Sumatoria = "Sumatoria: "+ str(suma)
        plt.text(8,75, Sumatoria)
        for exponente in resultados_pv_y_lyap[archivo][serie]['lyap']['exp']:
            if exponente > 0 and resultados_pv_y_lyap[archivo][serie]['lyap']['sum']<0:
                plt.text(9,70, "Es caotico")
                break
        if serie == 'V':
            media = np.format_float_scientific(resultados_kolgomorov[archivo]['com_M_V'], unique=False, precision=4)
            mediana = np.format_float_scientific(resultados_kolgomorov[archivo]['com_m_V'], unique=False, precision=4)
            ck = f"Complejidades Kolgomorov: media {media} mediana {mediana}"
        else:
            media = np.format_float_scientific(resultados_kolgomorov[archivo]['com_M_T'], unique=False, precision=4)
            mediana = np.format_float_scientific(resultados_kolgomorov[archivo]['com_m_T'], unique=False, precision=4)
            ck = f"Complejidades Kolgomorov: media {media} mediana {mediana}"
        plt.text(9,65, ck)
        plt.ylim((0,102))
        plt.savefig(f"./GraficosFV/{archivo}{serie}.png")
        plt.clf()

DATA_20190307-0004_9_10Millones
DATA_20190307-0002_7_10Millones
DATA_20190307-0002_6_10Millones
DATA_20190307-0002_1_10Millones
DATA_20190307-0004_3_10Millones
DATA_20190307-0004_5_10Millones
DATA_20190307-0004_4_10Millones
DATA_20190307-0003_5_28Millones
DATA_20190307-0002_2_10Millones
DATA_20190307-0003_8_10Millones
DATA_20190307-0003_4_37Millones
DATA_20190307-0004_6_10Millones
DATA_20190307-0004_1_10Millones
DATA_20190307-0003_8_37Millones
DATA_20190307-0002_9_10Millones
DATA_20190307-0003_4_10Millones
DATA_20190307-0003_5_10Millones
DATA_20190307-0004_2_10Millones
DATA_20190307-0004_8_10Millones
DATA_20190307-0004_7_10Millones
DATA_20190307-0002_4_10Millones
DATA_20190307-0002_5_10Millones
DATA_20190307-0002_8_10Millones
DATA_20190307-0002_3_10Millones


<Figure size 864x504 with 0 Axes>

In [149]:
resultados_kolgomorov['DATA_20190307-0004_9_10Millones']['com_M_V']

1.019483652827443

In [153]:
def check_dE(fvs):
    caotico = False
    neecsito_ver_mas = True
    fv_anterior = 100
    for i,fv in enumerate(fvs):
        if fv < 0.01:
            caotico = True
            neecsito_ver_mas = False
            break
        else:
            if fv_anterior == 100:
                fv_anterior = fv
            else:
                variacionfv = 1 - fv/fv_anterior
                if variacionfv < -0.1:
                    caotico = False
                    neecsito_ver_mas = False
                    break
                elif variacionfv < 0.01 and fv <= 0.05:
                    caotico = True
                    neecsito_ver_mas = False
                    break
                fv_anterior = fv
    if caotico:
        return i+1
    else:
        return False